In [1]:
#mandatory imports
import glob
import os
import unicodedata
import io
import string
import pandas as pd

In [2]:
all_letters = string.ascii_letters +".,;':"
print(all_letters)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,;':


In [3]:
print(len(all_letters))

57


In [4]:
#helper fuinctions -https://www.kaggle.com/code/sinclairg/pytorch-rnn-name-classification
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [5]:
surname_class = {}
origins = []
for filename in glob.glob('/home/maximus1/Downloads/archive (1)/data/names/*.txt'):
    origin = filename.split('/')[-1].split('.')[0]
    origins.append(origin)
    lines = readLines(filename)
    for surname in lines:
        surname_class[surname] = origin

print(len(surname_class))

17416


In [6]:
print("{} different origins are available".format(len(origins)))

18 different origins are available


In [7]:
surname_class

{'Aalsburg': 'Dutch',
 'Aalst': 'Dutch',
 'Aarle': 'Dutch',
 'Achteren': 'Dutch',
 'Achthoven': 'Dutch',
 'Adrichem': 'Dutch',
 'Aggelen': 'Dutch',
 'Agteren': 'Dutch',
 'Agthoven': 'Dutch',
 'Akkeren': 'Dutch',
 'Aller': 'Dutch',
 'Alphen': 'Dutch',
 'Alst': 'Dutch',
 'Altena': 'Dutch',
 'Althuis': 'Dutch',
 'Amelsvoort': 'Dutch',
 'Amersvoort': 'Dutch',
 'Amstel': 'Dutch',
 'Andel': 'Dutch',
 'Andringa': 'Dutch',
 'Ankeren': 'Dutch',
 'Antwerp': 'Dutch',
 'Antwerpen': 'Dutch',
 'Apeldoorn': 'Dutch',
 'Arendonk': 'Dutch',
 'Asch': 'Dutch',
 'Assen': 'Dutch',
 'Baarle': 'Dutch',
 'Bokhoven': 'Dutch',
 'Breda': 'Italian',
 'Bueren': 'Dutch',
 'Buggenum': 'Dutch',
 'Buiren': 'Dutch',
 'Buren': 'Dutch',
 'Can': 'Dutch',
 'Cann': 'English',
 'Canne': 'Dutch',
 'Daal': 'Dutch',
 'Daalen': 'Dutch',
 'Dael': 'Dutch',
 'Daele': 'Dutch',
 'Dale': 'English',
 'Dalen': 'Dutch',
 'Laar': 'Dutch',
 'Vliert': 'Dutch',
 'Akker': 'Dutch',
 'Denend': 'Dutch',
 'Aart': 'Dutch',
 'Beek': 'Dutch',
 'Berg'

In [8]:
dataset = pd.DataFrame.from_dict(surname_class, orient="index").reset_index()
dataset.columns = ['surname', 'origin']

In [9]:
dataset

,surname,origin
0,Aalsburg,Dutch
1,Aalst,Dutch
2,Aarle,Dutch
3,Achteren,Dutch
4,Achthoven,Dutch
...,...,...
17411,Younis,English
17412,Youssouf,English
17413,Yule,English
17414,Yusuf,English


In [10]:
# pytorch imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [11]:
#creating dataset
class NameDataset(Dataset):
    def __init__(self,dataframe):
        self.dataframe = dataframe
        self.origin = self.dataframe['origin'].values.tolist()
        self.origin_list = list(sorted(set(self.origin)))
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, index):
        surname = self.dataframe['surname'][index]
        origin = self.dataframe['origin'][index]
        item = {'surname':surname, 'origin':origin}
        return item
    
    def get_origins(self):
        return self.origin_list

    def origin_id(self,origin):
        return self.origin_list.index(origin)
    
    def get_origin(self,id):
        return self.origin_list[id] 

In [12]:
training_dataset = NameDataset(dataset)
training_dataset[0]

{'surname': 'Aalsburg', 'origin': 'Dutch'}

In [13]:
print(training_dataset.get_origin(1))
print(training_dataset.get_origin(0))

Chinese
Arabic


In [14]:
print(training_dataset.get_origins())

['Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese']


In [15]:
#creating data loader
dataloader = DataLoader(training_dataset, batch_size=4096,shuffle=True)

In [16]:
for batch in dataloader:
    print(batch['surname'], batch['origin'])
    break

['Samaha', 'Forster', 'Abbey', 'Kilner', 'Atamanenko', 'Turtsevich', 'Mcintyre', 'Ageichev', 'Kaminaga', 'Docherty', 'Beyersdorf', 'Satoya', 'Tchanchikov', 'Wojewodka', 'Evelson', 'Zhiganov', 'Bureau', 'Ungaro', 'Chaleev', 'Chromy', 'Agrikov', 'Sharp', 'Vavkin', 'Avkhimovitch', 'Nassar', 'Dobronos', 'Phelan', 'Trampotova', 'Alagona', 'Meeuwissen', 'Allard', 'Gesse', 'Maksimychev', 'Strobel', 'Mikhalitsin', 'Yankelevich', 'Nader', 'Benson', 'Rawsthorne', 'Ajello', 'Steer', 'Zhurko', 'Leroy', 'Griffiths', 'Agranowsky', 'Rainov', 'Loong', 'Achthoven', 'Togoev', 'Mihilev', 'Sevriens', 'Aliprandi', 'Baikin', 'Shaughnessy', 'Talyzin', 'Evdakov', 'Yamakage', 'Janin', 'Yarwood', 'Norburn', 'Legg', 'Aldersley', 'Anuriev', 'Saleh', 'Zhelezny', 'Anwar', 'Shalin', 'Zabala', 'Balakhnin', 'Omura', 'Abdulhabirov', 'Dowler', 'Zhirmunsky', 'Tompkins', 'Vipper', 'Lazhentsev', 'Mancuso', 'Zhitetsky', 'Abdullaev', 'Tunev', 'Griff', 'Bakhrakh', 'Davies', 'Fenn', 'Abashidze', 'Onopriev', 'Antyuhov', 'Zhumen

In [17]:
# Hyperparams
HIDDEN_SIZE = 100
N_LAYERS = 4
BATCH_SIZE = 256
N_EPOCHS = 100
N_CHARS = 128  # ASCII

In [18]:
def origin_to_tensor(origins):
    origin_ids = [training_dataset.origin_id(
        origin) for origin in origins]
    return torch.LongTensor(origin_ids)

def str2ascii_arr(msg):
    arr = [ord(c) for c in msg]
    return arr, len(arr)

In [19]:
# pad sequences 
def pad_sequences(vectorized_seqs,seq_lens, origins):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lens.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lens)):
        seq_tensor[idx,:seq_len] = torch.LongTensor(seq)

    # sorting tensors by length
    seq_lens, perm_idx = seq_lens.sort(0, descending=True)
    seq_tensor = seq_tensor[perm_idx]
    #print(seq_tensor)

    #sorting the target in same order
    #print(origins)
    target = origin_to_tensor(origins)
    if len(origins):
        target = target[perm_idx]

    return seq_tensor,seq_lens,target

In [20]:
def generate_io(surnames,origins):
    sequence_and_len = [str2ascii_arr(surname) for surname in surnames]
    vectorized_sequence = [item[0] for item in sequence_and_len]
    seq_lens = torch.LongTensor([item[1] for item in sequence_and_len])
    #print(vectorized_sequence, seq_lens)
    return (pad_sequences(vectorized_sequence,seq_lens, origins))

In [21]:
training_dataset.__getitem__(1)

{'surname': 'Aalst', 'origin': 'Dutch'}

In [22]:
for i, batch in enumerate(dataloader):
    surname , origin = batch['surname'],batch['origin'] 
    seq_tensor, seq_lens, target = generate_io(surname,origin)
    print(seq_tensor,seq_lens,target)
    if i ==3:
        break

tensor([[ 83, 104, 105,  ..., 116, 111, 118],
        [ 75,  97, 116,  ...,   0,   0,   0],
        [ 65,  98, 100,  ...,   0,   0,   0],
        ...,
        [ 83, 111,   0,  ...,   0,   0,   0],
        [ 68, 111,   0,  ...,   0,   0,   0],
        [ 75, 105,   0,  ...,   0,   0,   0]]) tensor([19, 16, 15,  ...,  2,  2,  2]) tensor([14, 14, 14,  ..., 11, 17, 10])
tensor([[ 66, 101, 107,  ..., 115, 107, 121],
        [ 77,  97,  99,  ..., 110,  97,   0],
        [ 67, 104, 114,  ..., 115,   0,   0],
        ...,
        [ 86, 111,   0,  ...,   0,   0,   0],
        [ 66,  97,   0,  ...,   0,   0,   0],
        [ 65, 110,   0,  ...,   0,   0,   0]]) tensor([18, 17, 16,  ...,  2,  2,  2]) tensor([14,  8,  7,  ..., 17,  0, 17])
tensor([[ 66, 101, 107,  ..., 104, 101, 118],
        [ 86,  97, 110,  ...,  99, 107,   0],
        [ 83, 104,  97,  ..., 115,   0,   0],
        ...,
        [ 74, 111,   0,  ...,   0,   0,   0],
        [ 85, 105,   0,  ...,   0,   0,   0],
        [ 76, 101,   

In [31]:
class RNNClassifier(nn.Module):

    def __init__(self,input_size,hidden_size, output_size,n_layers=1,bidirectional=True):
        super().__init__()
        self.hidden_size=hidden_size
        self.n_layers = n_layers
        self.n_directions = int(bidirectional)+1

        self.embedding_layer = nn.Embedding(input_size,hidden_size)
        self.gru_layer = nn.GRU(hidden_size, hidden_size,n_layers,bidirectional=bidirectional)
        self.fc_layer = nn.Linear(hidden_size,output_size)

    def forward(self,input,seq_lens):
        # we shall run this all at once 
        # initial input shape = B,S 
        # transpose= Seq, Batch
        input = input.t()
        batch = input.size(1)

        hidden = self._init_hidden(batch).cuda()
        # embedding = S,B to S, B, I (embedding sie)
        embedding = self.embedding_layer(input)

        #packing them
        gru_input = pack_padded_sequence(embedding,seq_lens.data.cpu().numpy())

        self.gru_layer.flatten_parameters()
        output,hidden = self.gru_layer(gru_input,hidden)

        # using last layer output as FC input
        fc_output = self.fc_layer(hidden[-1])
        return fc_output

    def _init_hidden(self,batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions, batch_size, self.hidden_size)
        return hidden

In [32]:
N_ORIGINS=18
model = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_ORIGINS, N_LAYERS)

In [33]:
model

RNNClassifier(
  (embedding_layer): Embedding(128, 100)
  (gru_layer): GRU(100, 100, num_layers=4, bidirectional=True)
  (fc_layer): Linear(in_features=100, out_features=18, bias=True)
)

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cpu


In [ ]:
total_loss =0
num_epochs=1000
optimizer = torch.optim.SGD(model.parameters(),lr=0.000000001) #tried with ADAM too.
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    for i, batch  in enumerate(dataloader):
        surname = batch['surname']
        origin = batch['origin']
        #generating padded inputs-outputs
        input,seq_lens,target = generate_io(surname, origin)
        #sending to device
        input = input
        seq_lens = seq_lens
        target = target
        #generating output and calculating loss
        output = model(input,seq_lens)
        loss = criterion(output,target)
        total_loss+=loss.item()
        #resetting grads, backpropagating and updating weights
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    
    print("Epoch - {}, Loss - {}".format(epoch,total_loss))